In [7]:
import pandas as pd

In [8]:
data = pd.read_csv("dataset_2019_2022.csv")
df = data.copy() #Create a backup of the original dataset

In [10]:
# Checking for missing values and data types
missing_values = df.isnull().sum()
data_types = df.dtypes

print("Missing Values:\n", missing_values,"\n")
print("Data Types:\n", data_types)
df.columns


Missing Values:
 customer_id         0
product_id          0
basket_id           0
loyalty             0
household_type      0
age_band            0
department          0
brand               0
commodity           0
store               0
price               0
transaction_date    0
dtype: int64 

Data Types:
 customer_id           int64
product_id            int64
basket_id             int64
loyalty              object
household_type       object
age_band             object
department           object
brand                object
commodity            object
store                 int64
price               float64
transaction_date     object
dtype: object


Index(['customer_id', 'product_id', 'basket_id', 'loyalty', 'household_type',
       'age_band', 'department', 'brand', 'commodity', 'store', 'price',
       'transaction_date'],
      dtype='object')

In [38]:
# Check for duplicate rows
duplicate_rows = df.duplicated().sum()
print(duplicate_rows)
# Remove duplicates if any
if duplicate_rows > 0:
    df = df.drop_duplicates()

duplicate_rows = df.duplicated().sum()
print(duplicate_rows)

59
0


In [39]:
# Analyzing Commodity-Department Relationships
commodity_department_relationships = df.groupby(['commodity', 'department']).size().unstack(fill_value=0)
commodity_department_relationships.head()
# Identifying commodities listed under multiple departments
commodities_multiple_departments = commodity_department_relationships[(commodity_department_relationships > 0).sum(axis=1) > 1]

commodities_multiple_departments  # Displaying the first few rows for inspection


department,Cosmetics,Deli,Floral,Grocery,Meat,Nutrition,Pastry,Pharmaceutical,Produce,Salad Bar,Seafood
commodity,,,,,,,,,,,
Baby foods,0,0,0,0,0,40,0,318,0,0,0
Bread,0,0,0,0,0,16,1240,0,0,0,0
Cheese,0,795,0,1267,0,0,0,0,0,0,0
Condiments,0,0,0,301,0,36,0,0,0,0,0
Cookies,0,0,0,390,0,0,408,0,0,0,0
Coupon/misc items,20,0,0,19,0,0,0,0,0,0,0
Frozen meat,0,0,0,1028,560,0,0,0,0,0,0
Meat - Other,0,0,0,273,536,0,0,0,0,0,0
Prepared food,0,260,0,0,0,92,0,0,0,0,0


In [40]:
# Function to determine the department with the highest frequency for a commodity
def get_primary_department(commodity_row):
    return commodity_row.idxmax()

# Applying the function to each commodity listed under multiple departments
primary_departments = commodities_multiple_departments.apply(get_primary_department, axis=1)

# Updating the dataset: For each commodity, set its department to the primary department
for commodity, primary_dept in primary_departments.items():
    df.loc[df['commodity'] == commodity, 'department'] = primary_dept


In [41]:
# Example: Saving the DataFrame 'data' to a CSV file
file_name = "clean_dataset_2019_2022.csv"  # You can specify a full file path here
df.to_csv(file_name, index=False)  # Setting index=False to not save row indices

In [42]:
df.head()

,customer_id,product_id,basket_id,loyalty,household_type,age_band,department,brand,commodity,store,price,transaction_date,year,month,day_of_week,transaction_year_month
0,15803,1131974,57266,Loyalist,1 adult with kids,19-24,Grocery,private,Baked bread/buns/rolls,374,0.99,2020-10-05,2020,10,0,2020-10
1,15803,1051516,57266,Loyalist,1 adult with kids,19-24,Produce,national,Vegetables - all others,374,0.70,2020-10-24,2020,10,5,2020-10
2,15803,967254,57266,Loyalist,1 adult with kids,19-24,Pharmaceutical,national,Cold and flu,374,1.68,2020-10-18,2020,10,6,2020-10
3,15803,1134222,57266,Loyalist,1 adult with kids,19-24,Grocery,private,Paper housewares,374,2.59,2020-10-23,2020,10,4,2020-10
4,15803,1003421,57266,Loyalist,1 adult with kids,19-24,Grocery,national,Soup,374,0.60,2020-10-27,2020,10,1,2020-10


In [5]:
# Convert transaction_date to datetime
df['transaction_date'] = pd.to_datetime(df['transaction_date'], dayfirst=True)

# Extract year, month, and day of the week from transaction_date
df['year'] = df['transaction_date'].dt.year
df['month'] = df['transaction_date'].dt.month
df['day_of_week'] = df['transaction_date'].dt.dayofweek  # Monday=0, Sunday=6
df['transaction_year_month'] = df['transaction_date'].dt.to_period('M')

df.shape

(77750, 16)